In [ ]:
'''
Generate panels for figure 11 

Maximilian Eggl
Nov 2024
'''


import sys
    
from ExtFuncs import *
%load_ext autoreload
%autoreload 2

import scipy.stats as stats

plt.rc('text', usetex=True)
font = {'family' : 'serif',
        'size'   : 14}
plt.rc('font', **font)
plt.rc('ytick', labelsize=28) 
plt.rc('xtick', labelsize=28)

import networkx as nx
import networkx.algorithms.community as nx_comm
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import ward, fcluster
from scipy.spatial.distance import squareform,pdist

import matplotlib.patches as patches
import matplotlib.colors as mcolors

plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False

from statsmodels.stats.multitest import multipletests

In [ ]:
datDir = './Dat/'
ColorCode = {'D1Nac':np.array([117/255,20/255,124/255]),'D2Nac':np.array([241/255,164/255,84/255]),
             'mPFC':np.array([86/255,187/255,126/255]),'Sus':np.array((0.9682, 0.7208441, 0.612293)),
            'Res':np.array((0.705673158, 0.01555616, 0.150232812)),'nForg':np.array((0.6672529243333334, 0.779176457, 0.992959213)),
            'Forg':np.array((0.2298057, 0.298717966, 0.753683153))}

FigLabs = ['Mean degree \n [edges per node]','Efficiency','Modularity','Betweeness','Assortativity']

def PlotSig(left,right,height,starAdjust=0.02,col = 'k',ax=None, text = None):
    if(ax is None):
        plt.plot([left,right],[height]*2,c=col)
        if(text is None):
            plt.plot((left+right)/2,height+starAdjust,marker=(6, 2,0 ),ms=10,c=col)
        else:
            plt.text((left+right)/2,height+starAdjust,text,ha='center')
    else:
        ax.plot([left,right],[height]*2,c=col)
        if(text is None):
            ax.plot((left+right)/2,height+starAdjust,marker=(6, 2,0 ),ms=10,c=col)
        else:
            ax.text((left+right)/2,height+starAdjust,text,ha='center')   

def PlotViolScat(A,col,pos,lab=None,ax= None,hatch=False):
    if(ax is None):
        vp = plt.violinplot(A,
                positions=[pos],widths=0.2,showmeans='True'
                )  
        vp['bodies'][0].set_facecolor(col)
        vp['cbars'].set_color(col)
        vp['cmins'].set_color(col)
        vp['cmaxes'].set_color(col)
        vp['cmeans'].set_color('black')
        plt.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)
        if(hatch):
            vp['bodies'][0].set_hatch('//')
    else:
        vp = ax.violinplot(A,
                positions=[pos],widths=0.2,showmeans='True'
                )  
        vp['bodies'][0].set_facecolor(col)
        vp['cbars'].set_color(col)
        vp['cmins'].set_color(col)
        vp['cmaxes'].set_color(col)
        vp['cmeans'].set_color('black')
        if(hatch):
            vp['bodies'][0].set_hatch('//')
        ax.scatter(pos*np.ones(len(A))+(np.random.rand(len(A))-0.5)*0.1,A,
                color=col,label=lab)

def GetGraphMetrics(G,labels):
    #degree mean
    mdd = np.mean(sorted((d for n, d in G.degree()), reverse=True))
    #global efficiency
    ge  = nx.global_efficiency(G)
    #betweenness
    be  = np.mean(list(nx.betweenness_centrality(G,normalized=True).values()))
    #modularity
    try: #this messes stuff up - without try gets the figures in E
        t = [np.argwhere(labels==i).squeeze() for i in range(1,max(labels)+1)]
        mod = nx_comm.modularity(G, t)
    except:
        mod = 0
    #assortativity
    ass = nx.degree_pearson_correlation_coefficient(G)
    return mdd,ge,be,mod,ass

In [ ]:
DatA,TimesA,DatNamesA,TimeCalcDatA,CalcDatA,ShockTimesA,cRawA = GetDat('Batch A 2022 mPFC',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],datDir)
DatB,TimesB,DatNamesB,TimeCalcDatB,CalcDatB,ShockTimesB,cRawB = GetDat('Batch B 2022 mPFC',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],datDir)

DatMPC = DatA+DatB
TimesMPC = np.vstack([TimesA,TimesB])
DatNamesMPC = DatNamesA + DatNamesB
TimeCalcDatMPC = TimeCalcDatA+TimeCalcDatB
CalcDatMPC = CalcDatA+CalcDatB
ShockTimesMPC = np.vstack([ShockTimesA,ShockTimesB])
labelsMPC = np.array([1, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1])

In [ ]:
DatC,TimesC,DatNamesC,TimeCalcDatC,CalcDatC,ShockTimesC,rawC = GetDat('Batch C 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=1)
DatD,TimesD,DatNamesD,TimeCalcDatD,CalcDatD,ShockTimesD,rawD = GetDat('Batch D 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,eFlag=1)
DatNAD1 = DatC+DatD
TimesNAD1 = np.vstack([TimesC,TimesD])
DatNamesNAD1 = DatNamesC + DatNamesD
TimeCalcDatNAD1 = TimeCalcDatC+TimeCalcDatD
CalcDatNAD1 = CalcDatC+CalcDatD
ShockTimesNAD1 = np.vstack([ShockTimesC,ShockTimesD])
RawDatNAD1 = rawC+rawD
labelsNAD1 = np.array([1, 2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2])

In [ ]:
DatE,TimesE,DatNamesE,TimeCalcDatE,CalcDatE,ShockTimesE,rawE = GetDat('Batch E 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,1,eFlag=1)
DatF,TimesF,DatNamesF,TimeCalcDatF,CalcDatF,ShockTimesF,rawF = GetDat('Batch F 2022 NAc',['2.PTSD/','3.Reminder 1/','4.Reminder 2/','5.Reminder 3/','7.Reminder 4/'],
                                                                 datDir,1,eFlag=1)
DatNAD2 = DatE+DatF
TimesNAD2 = np.vstack([TimesE,TimesF])
DatNamesNAD2 = DatNamesE + DatNamesF
TimeCalcDatNAD2 = TimeCalcDatE+TimeCalcDatF
CalcDatNAD2 = CalcDatE+CalcDatF
ShockTimesNAD2 = np.vstack([ShockTimesE,ShockTimesF])
RawDatNAD2 = rawE+rawF
labelsNAD2 = np.array([2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2])

In [ ]:
FullOrder = []
FullOrderA = []
i=0
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    TC = TimeCalcDatMPC[j][i]
    ST = ShockTimesMPC[j][i]
    Order = np.zeros(Calc.shape[0])
    Order2 = np.zeros(Calc.shape[0])
    Fire0 = Calc[:,TC<ST[0]].mean(axis=1)==0.0
    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order = np.zeros_like(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    FullOrder.append(Order)
    FullOrderA.append(Order2)
    
print('-'*10 + 'Done 1'+'-'*10)    
i = 1
FullOrder2 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrder[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderA[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder2.append(Order)
print('-'*10 + 'Done 2'+'-'*10)
i = 2
FullOrder3 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder3.append(Order)
print('-'*10 + 'Done 3'+'-'*10)
i = 3
FullOrder4 = []
for j in range(len(CalcDatMPC)):
    Calc = CalcDatMPC[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrder3[j]<0,FullOrder2[j]<0,FullOrder[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrder3[j]>0,FullOrder2[j]>0,FullOrderA[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrder4.append(Order)
print('-'*10 + 'Done 4'+'-'*10)
TotalOrder = [(F1<0)+(F2<0)+(F3<0)+(F4<0) for F1,F2,F3,F4 in
              zip(FullOrder,FullOrder2,FullOrder3,FullOrder4)]

In [ ]:
FullOrderD1 = []
FullOrderD1A = []
i=0
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    TC = TimeCalcDatNAD1[j][i]
    ST = ShockTimesNAD1[j][i]

    SV = np.zeros_like(TC)
    A = np.argmin(np.abs(TC-ST[0]))
    B = np.argmin(np.abs(TC-ST[1]))

    SV[A:A+100] = 1
    SV[B:B+100] = 1

    MaxCorrV = []
    for C in Calc:
        MaxCorrV.append(np.corrcoef(SV,C)[0,1])
    MaxCorrV = np.array(MaxCorrV)

    Order = np.zeros(Calc.shape[0]).astype(np.float64)
    Order2 = np.zeros(Calc.shape[0]).astype(np.float64)
    Fire0 = MaxCorrV>0.2


    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    Order2[Order<0] = 0

    FullOrderD1.append(Order)
    FullOrderD1A.append(Order2)

print('1 done')

i = 1
FullOrderD12 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrderD1[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderD1A[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD12.append(Order)

print('2 done')

i = 2
FullOrderD13 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD12[j]<0,FullOrderD1[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD12[j]>0,FullOrderD1A[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD13.append(Order)
    
print('3 done')

i = 3
FullOrderD14 = []
for j in range(len(CalcDatNAD1)):
    Calc = CalcDatNAD1[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD13[j]<0,FullOrderD12[j]<0,FullOrderD1[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD13[j]>0,FullOrderD12[j]>0,FullOrderD1A[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD14.append(Order)
    
print('4 done')

TotalOrderD1 = [(F1<0)+(F2<0)+(F3<0)+(F4<0) for F1,F2,F3,F4 in
              zip(FullOrderD1,FullOrderD12,FullOrderD13,FullOrderD14)]

In [ ]:
FullOrderD2 = []
FullOrderD2A = []
i=0
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    TC = TimeCalcDatNAD2[j][i]
    ST = ShockTimesNAD2[j][i]

    SV = np.zeros_like(TC)
    A = np.argmin(np.abs(TC-ST[0]))
    B = np.argmin(np.abs(TC-ST[1]))

    SV[A:A+100] = 1
    SV[B:B+100] = 1

    MaxCorrV = []
    for C in Calc:
        MaxCorrV.append(np.corrcoef(SV,C)[0,1])
    MaxCorrV = np.array(MaxCorrV)

    Order = np.zeros(Calc.shape[0]).astype(np.float64)
    Order2 = np.zeros(Calc.shape[0]).astype(np.float64)
    Fire0 = MaxCorrV>0.2


    MaxCorrV = []
    for kk,C1 in enumerate(Calc):
        maxCorr = 0
        for C2 in Calc[Fire0]:
            maxCorr = max(np.corrcoef(C1,C2)[0,1],maxCorr)
        MaxCorrV.append(maxCorr)
        if stats.ttest_ind(C1[TC<ST[0]],C1[TC>ST[0]],alternative='greater').pvalue< 0.05:
            Order2[kk] = 1
    MaxCorrV = np.array(MaxCorrV)
    Order[MaxCorrV>0.5] = -1
    Order[MaxCorrV==1] = -2
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
    Order2[np.isnan(np.sum(Calc,axis=1))] = math.nan
    
    Order2[Order<0] = 0

    FullOrderD2.append(Order)
    FullOrderD2A.append(Order2)

print('1 done')

i = 1
FullOrderD22 = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for C1 in Calc:
        maxCorr = []
        for C2 in Calc[FullOrderD2[j]<0]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[FullOrderD2A[j]>0]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD22.append(Order)

print('2 done')

i = 2
FullOrderD23 = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD22[j]<0,FullOrderD2[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD22[j]>0,FullOrderD2A[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD23.append(Order)
    
print('3 done')

i = 3
FullOrderD24 = []
for j in range(len(CalcDatNAD2)):
    Calc = CalcDatNAD2[j][i]
    MaxCorrV = []
    MaxCorrV2 = []
    for ll,C1 in enumerate(Calc):
        maxCorr = []
        for C2 in Calc[np.logical_or(FullOrderD23[j]<0,FullOrderD22[j]<0,FullOrderD2[j]<0)]:
            if(np.isnan(C2).any()):
                maxCorr.append(math.nan)
            else:
                maxCorr.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV.append(maxCorr)
        maxCorr2 = []
        for C2 in Calc[np.logical_or(FullOrderD23[j]>0,FullOrderD22[j]>0,FullOrderD2A[j]>0)]:
            if(np.isnan(C2).any()):
                maxCorr2.append(math.nan)
            else:
                maxCorr2.append(np.corrcoef(C1,C2)[0,1])
        MaxCorrV2.append(maxCorr2)

    MaxCorrV = np.array(MaxCorrV)
    MaxCorrV = np.nanmax(MaxCorrV,axis=-1)

    MaxCorrV2 = np.array(MaxCorrV2)
    MaxCorrV2 = np.nanmax(MaxCorrV2,axis=-1)

    Order = np.zeros_like(MaxCorrV)
    for ll,(M1,M2) in enumerate(zip(MaxCorrV,MaxCorrV2)):
        if M1 > 0.5:
            Order[ll] = -1
        elif M2 > 0.5 and M1 < 0.5:
            Order[ll] = 1
    Order[np.isnan(np.sum(Calc,axis=1))] = math.nan
        
    FullOrderD24.append(Order)
    
print('4 done')

TotalOrderD2 = [(F1<0)+(F2<0)+(F3<0)+(F4<0) for F1,F2,F3,F4 in
              zip(FullOrderD2,FullOrderD22,FullOrderD23,FullOrderD24)]

In [ ]:
GM = np.zeros((len(CalcDatMPC),5,5))
for i,C1 in enumerate(CalcDatMPC):
    for j,C2 in enumerate(C1):

        Calc = C2[~np.isnan(C2.sum(axis=1))]
        Calc = Calc[Calc.sum(axis=1)>0]
        M = np.corrcoef(Calc)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'ward')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        try:
            GM[i,j,:] = GetGraphMetrics(G,labels)
        except:
            GM[i,j,:] = math.nan
GMD1 = np.zeros((len(CalcDatNAD1),5,5))
for i,C1 in enumerate(CalcDatNAD1):
    for j,C2 in enumerate(C1):
        Calc = C2[~np.isnan(C2.sum(axis=1))]
        Calc = Calc[Calc.sum(axis=1)>0]
        M = np.round(np.corrcoef(Calc),6)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'complete')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        GMD1[i,j,:] = GetGraphMetrics(G,labels)
GMD2 = np.zeros((len(CalcDatNAD2),5,5))
for i,C1 in enumerate(CalcDatNAD2):
    for j,C2 in enumerate(C1):
        Calc = C2[~np.isnan(C2.sum(axis=1))]
        Calc = Calc[Calc.sum(axis=1)>0]
        M = np.corrcoef(Calc)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'ward')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        GMD2[i,j,:] = GetGraphMetrics(G,labels)

In [ ]:
i=0
plt.errorbar([1,2,3,4],np.nanmean(GM[:,:4,i],axis=0),stats.sem(GM[:,:4,i],axis=0,nan_policy='omit'),
             lw=3,label='mPFC',c=ColorCode['D1Nac'])
plt.errorbar([1,2,3,4],GMD1[:,:4,i].mean(axis=0),stats.sem(GMD1[:,:4,i],axis=0),lw=3,c=ColorCode['D2Nac'],
             label='NAc D1')
plt.errorbar([1,2,3,4],GMD2[:,:4,i].mean(axis=0),stats.sem(GMD2[:,:4,i],axis=0),lw=3,c=ColorCode['mPFC'],label='NAc D2')
plt.ylabel(FigLabs[i],fontsize=32)
plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
#if i == 0: plt.legend(fontsize=24,frameon=False)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)  
PlotSig(1,2,57,1,col=ColorCode['mPFC'])
plt.plot((1,1),(15,20),'k')
plt.plot((1,1),(25,30),'k')
plt.plot([0.95,1.05],[22.25,22.25],marker=(6, 2,0 ),ms=10,c='k',linestyle='')

plt.plot((2,2),(18,26.5),'k')
plt.plot((2,2),(31.5,40),'k')
plt.plot([1.85,1.95,2.05,2.15],[29]*4,marker=(6, 2,0 ),ms=10,c='k',linestyle='')

plt.plot((3,3),(20,29.5),'k')
plt.plot((3,3),(34.5,42),'k')
plt.plot([2.85,2.95,3.05,3.15],[32]*4,marker=(6, 2,0 ),ms=10,c='k',linestyle='')

plt.plot((4,4),(20,26.5),'k')
plt.plot((4,4),(31.5,38),'k')
plt.plot([3.85,3.95,4.05,4.15],[29]*4,marker=(6, 2,0 ),ms=10,c='k',linestyle='')
plt.legend(fontsize=24,frameon=False,ncols=3,columnspacing=0.8,loc=1,
           bbox_to_anchor=(1.15,1.2),handletextpad=0.1)

In [ ]:
i=1
plt.errorbar([1,2,3,4],np.nanmean(GM[:,:4,i],axis=0),stats.sem(GM[:,:4,i],axis=0,nan_policy='omit'),
             lw=3,label='mPFC',c=ColorCode['D1Nac'])
plt.errorbar([1,2,3,4],GMD1[:,:4,i].mean(axis=0),stats.sem(GMD1[:,:4,i],axis=0),lw=3,c=ColorCode['D2Nac'],
             label='NAc D1')
plt.errorbar([1,2,3,4],GMD2[:,:4,i].mean(axis=0),stats.sem(GMD2[:,:4,i],axis=0),lw=3,c=ColorCode['mPFC'],label='NAc D2')
plt.ylabel(FigLabs[i],fontsize=32)
plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
#if i == 0: plt.legend(fontsize=24,frameon=False)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)  
#PlotSig(1,2,0.51,0.01,col=ColorCode['D1Nac'])


plt.plot((1,2),(0.51,0.51),c=ColorCode['D1Nac'])
plt.plot(np.array([0.85,0.95,1.05,1.15])+0.5,[0.52]*4,marker=(6, 2,0 ),ms=10,c=ColorCode['D1Nac'],linestyle='')

plt.plot((1,2),(0.68,0.68),c=ColorCode['mPFC'])
plt.plot(np.array([0.95,1.05])+0.5,[0.69]*2,marker=(6, 2,0 ),ms=10,c=ColorCode['mPFC'],linestyle='')

plt.plot((1,2),(0.72,0.72),c=ColorCode['D2Nac'])
plt.plot(np.array([0.95,1.05])+0.5,[0.73]*2,marker=(6, 2,0 ),ms=10,c=ColorCode['D2Nac'],linestyle='')


plt.plot((1,1),(0.4,0.43),'k')
plt.plot((1,1),(0.47,0.5),'k')
plt.plot(np.array([0.85,0.95,1.05,1.15]),[0.45]*4,marker=(6, 2,0 ),ms=10,c='k',linestyle='')

plt.plot((2,2),(0.52,0.55),'k')
plt.plot((2,2),(0.59,0.62),'k')
plt.plot(np.array([0.85,0.95,1.05,1.15])+1,[0.57]*4,marker=(6, 2,0 ),ms=10,c='k',linestyle='')

plt.plot((3,3),(0.52,0.55),'k')
plt.plot((3,3),(0.59,0.62),'k')
plt.plot(np.array([0.85,0.95,1.05,1.15])+2,[0.57]*4,marker=(6, 2,0 ),ms=10,c='k',linestyle='')

plt.plot((4,4),(0.52,0.55),'k')
plt.plot((4,4),(0.59,0.62),'k')
plt.plot(np.array([0.85,0.95,1.05,1.15])+3,[0.57]*4,marker=(6, 2,0 ),ms=10,c='k',linestyle='')

In [ ]:
i=3
plt.errorbar([1,2,3,4],np.nanmean(GM[:,:4,i],axis=0),stats.sem(GM[:,:4,i],axis=0,nan_policy='omit'),
             lw=3,label='mPFC',c=ColorCode['D1Nac'])
plt.errorbar([1,2,3,4],GMD1[:,:4,i].mean(axis=0),stats.sem(GMD1[:,:4,i],axis=0),lw=3,c=ColorCode['D2Nac'],
             label='NAc D1')
plt.errorbar([1,2,3,4],GMD2[:,:4,i].mean(axis=0),stats.sem(GMD2[:,:4,i],axis=0),lw=3,c=ColorCode['mPFC'],label='NAc D2')
plt.ylabel(FigLabs[i],fontsize=32)
plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
if i == 0: plt.legend(fontsize=24,frameon=False)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)  

plt.plot((1,1),(0.29,0.3),'k')
plt.plot((1,1),(0.33,0.34),'k')
plt.plot([0.95,1.05],[0.315,0.315],marker=(6, 2,0 ),ms=10,c='k',linestyle='')

plt.plot((2,2),(0.37,0.375),'k')
plt.plot((2,2),(0.392,0.4),'k')
plt.plot(np.array([0.95,1.05])+1,[0.385,0.385],marker=(6, 2,0 ),ms=10,c='k',linestyle='')

plt.plot((3,3),(0.35,0.38),'k')
plt.plot((3,3),(0.41,0.44),'k')
plt.plot(np.array([0.95,1.05])+2,[0.395,0.395],marker=(6, 2,0 ),ms=10,c='k',linestyle='')

plt.plot((4,4),(0.35,0.37),'k')
plt.plot((4,4),(0.40,0.41),'k')
plt.plot(np.array([0.95,1.05])+3,[0.385,0.385],marker=(6, 2,0 ),ms=10,c='k',linestyle='')

plt.plot((1,2),(0.5,0.5),c=ColorCode['mPFC'])
plt.plot(np.array([0.95,1.05])+0.5,[0.51]*2,marker=(6, 2,0 ),ms=10,c=ColorCode['mPFC'],linestyle='')

plt.plot((1,2),(0.53,0.53),c=ColorCode['D2Nac'])
plt.plot(np.array([0.95,1.05])+0.5,[0.54]*2,marker=(6, 2,0 ),ms=10,c=ColorCode['D2Nac'],linestyle='')

In [ ]:
i=4
plt.errorbar([1,2,3,4],np.nanmean(GM[:,:4,i],axis=0),stats.sem(GM[:,:4,i],axis=0,nan_policy='omit'),
             lw=3,label='mPFC',c=ColorCode['D1Nac'])
plt.errorbar([1,2,3,4],GMD1[:,:4,i].mean(axis=0),stats.sem(GMD1[:,:4,i],axis=0),lw=3,c=ColorCode['D2Nac'],
             label='NAc D1')
plt.errorbar([1,2,3,4],GMD2[:,:4,i].mean(axis=0),stats.sem(GMD2[:,:4,i],axis=0),lw=3,c=ColorCode['mPFC'],label='NAc D2')
plt.ylabel(FigLabs[i],fontsize=32)
plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
if i == 0: plt.legend(fontsize=24,frameon=False)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)  
PlotSig(1,2,0.4,0.01,col=ColorCode['D1Nac'])
PlotSig(1,2,0.36,0.01,col=ColorCode['D2Nac'])

plt.plot((1,2),(0.38,0.38),c=ColorCode['mPFC'])
plt.plot(np.array([0.95,1.05])+0.5,[0.39]*2,marker=(6, 2,0 ),ms=10,c=ColorCode['mPFC'],linestyle='')

plt.plot((1,1),(0.24,0.25),'k')
plt.plot((1,1),(0.27,0.28),'k')
plt.plot(1,0.26,marker=(6, 2,0 ),ms=10,c='k')

plt.plot((2,2),(0.152,0.16),'k')
plt.plot(2,0.17,marker=(6, 2,0 ),ms=10,c='k')

plt.plot((4,4),(0.155,0.16),'k')
plt.plot((4,4),(0.18,0.185),'k')
plt.plot(4,0.17,marker=(6, 2,0 ),ms=10,c='k')

In [ ]:
GMwo = np.zeros((len(CalcDatMPC),5,5))
for i,(C1,T) in enumerate(zip(CalcDatMPC,TotalOrder)):
    for j,C2 in enumerate(C1):

        Calc = C2[~T]
        Calc = Calc[Calc.sum(axis=1)>0]
        M = np.corrcoef(Calc)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'ward')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        try:
            GMwo[i,j,:] = GetGraphMetrics(G,labels)
        except:
            GMwo[i,j,:] = math.nan
            
GMD1wo = np.zeros((len(CalcDatNAD1),5,5))
for i,(C1,T) in enumerate(zip(CalcDatNAD1,TotalOrderD1)):
    for j,C2 in enumerate(C1):
        Calc = C2[~T]
        Calc = Calc[Calc.sum(axis=1)>0]
        M = np.round(np.corrcoef(Calc),6)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'complete')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        GMD1wo[i,j,:] = GetGraphMetrics(G,labels)
        
GMD2wo = np.zeros((len(CalcDatNAD2),5,5))
for i,(C1,T) in enumerate(zip(CalcDatNAD2,TotalOrderD2)):
    for j,C2 in enumerate(C1):
        Calc = C2[~T]
        Calc = Calc[Calc.sum(axis=1)>0]
        M = np.round(np.corrcoef(Calc),6)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'complete')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        GMD2wo[i,j,:] = GetGraphMetrics(G,labels)

In [ ]:
i=0
plt.errorbar([1,2,3,4],np.nanmean(GMwo[:,:4,i],axis=0),stats.sem(GMwo[:,:4,i],axis=0,nan_policy='omit'),
             lw=3,label='mPFC',c=ColorCode['D1Nac'],ls='--')
plt.errorbar([1,2,3,4],GMD1wo[:,:4,i].mean(axis=0),stats.sem(GMD1wo[:,:4,i],axis=0),lw=3,c=ColorCode['D2Nac'],
             label='NAc D1',ls='--')
plt.errorbar([1,2,3,4],GMD2wo[:,:4,i].mean(axis=0),stats.sem(GMD2wo[:,:4,i],axis=0),lw=3,c=ColorCode['mPFC'],
             label='NAc D2',ls='--')
plt.ylabel(FigLabs[i],fontsize=32)
plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
#if i == 0: plt.legend(fontsize=24,frameon=False)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)  
PlotSig(1,2,13,0.5,col=ColorCode['D1Nac'])
PlotSig(1,2,14,0.5,col=ColorCode['mPFC'])

plt.plot((3,3),(14,15),'k')
plt.plot(3,15.5,marker=(6, 2,0 ),ms=10,c='k')
plt.plot((4,4),(14,15),'k')
plt.plot(4,15.5,marker=(6, 2,0 ),ms=10,c='k')


In [ ]:
i=1
plt.errorbar([1,2,3,4],np.nanmean(GMwo[:,:4,i],axis=0),stats.sem(GMwo[:,:4,i],axis=0,nan_policy='omit'),
             lw=3,label='mPFC',c=ColorCode['D1Nac'],ls='--')
plt.errorbar([1,2,3,4],GMD1wo[:,:4,i].mean(axis=0),stats.sem(GMD1wo[:,:4,i],axis=0),lw=3,c=ColorCode['D2Nac'],
             label='NAc D1',ls='--')
plt.errorbar([1,2,3,4],GMD2wo[:,:4,i].mean(axis=0),stats.sem(GMD2wo[:,:4,i],axis=0),lw=3,c=ColorCode['mPFC'],
             label='NAc D2',ls='--')
plt.ylabel(FigLabs[i],fontsize=32)
plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
#if i == 0: plt.legend(fontsize=24,frameon=False)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)  


plt.plot((1,2),(0.6,0.6),c=ColorCode['D1Nac'])
plt.plot(np.array([0.95,1.05])+0.5,[0.61]*2,marker=(6, 2,0 ),ms=10,c=ColorCode['D1Nac'],linestyle='')

plt.plot((1,2),(0.65,0.65),c=ColorCode['mPFC'])
plt.plot(np.array([0.85,0.95,1.05,1.15])+0.5,[0.66]*4,marker=(6, 2,0 ),ms=10,c=ColorCode['mPFC'],linestyle='')

plt.plot((1,2),(0.625,0.625),c=ColorCode['D2Nac'])
plt.plot(np.array([0.95,1.05])+0.5,[0.635]*2,marker=(6, 2,0 ),ms=10,c=ColorCode['D2Nac'],linestyle='')



In [ ]:
i=3
plt.errorbar([1,2,3,4],np.nanmean(GMwo[:,:4,i],axis=0),stats.sem(GMwo[:,:4,i],axis=0,nan_policy='omit'),
             lw=3,label='mPFC',c=ColorCode['D1Nac'],ls='--')
plt.errorbar([1,2,3,4],GMD1wo[:,:4,i].mean(axis=0),stats.sem(GMD1wo[:,:4,i],axis=0),lw=3,c=ColorCode['D2Nac'],
             label='NAc D1',ls='--')
plt.errorbar([1,2,3,4],GMD2wo[:,:4,i].mean(axis=0),stats.sem(GMD2wo[:,:4,i],axis=0),lw=3,c=ColorCode['mPFC'],
             label='NAc D2',ls='--')
plt.ylabel(FigLabs[i],fontsize=32)
plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
#if i == 0: plt.legend(fontsize=24,frameon=False)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)  
plt.plot((4,4),(0.5,0.57),'k')
plt.plot([3.95,4.05],[0.6]*2,marker=(6, 2,0 ),ms=10,c='k',linestyle='')


In [ ]:
i=4
plt.errorbar([1,2,3,4],np.nanmean(GMwo[:,:4,i],axis=0),stats.sem(GMwo[:,:4,i],axis=0,nan_policy='omit'),
             lw=3,label='mPFC',c=ColorCode['D1Nac'],ls='--')
plt.errorbar([1,2,3,4],np.nanmean(GMD1wo[:,:4,i],axis=0),stats.sem(GMD1wo[:,:4,i],axis=0,nan_policy='omit'),lw=3,c=ColorCode['D2Nac'],
             label='NAc D1',ls='--')
plt.errorbar([1,2,3,4],GMD2wo[:,:4,i].mean(axis=0),stats.sem(GMD2wo[:,:4,i],axis=0),lw=3,c=ColorCode['mPFC'],
             label='NAc D2',ls='--')
plt.ylabel(FigLabs[i],fontsize=32)
plt.xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
#if i == 0: plt.legend(fontsize=24,frameon=False)
ax = plt.gca()
ax.spines["right"].set_visible(False)
ax.spines["top"].set_visible(False)  
plt.plot((4,4),(0.25,0.27),'k')
plt.plot([3.95,4.05],[0.3]*2,marker=(6, 2,0 ),ms=10,c='k',linestyle='')
plt.plot((3,3),(0.25,0.27),'k')
plt.plot(3,0.3,marker=(6, 2,0 ),ms=10,c='k')


In [ ]:
GM = np.zeros((len(CalcDatMPC),5,5))
for i,C1 in enumerate(CalcDatMPC):
    for j,C2 in enumerate(C1):

        Calc = C2[~np.isnan(C2.sum(axis=1))]
        Calc = Calc[Calc.sum(axis=1)>0]
        M = np.corrcoef(Calc)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'ward')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        try:
            GM[i,j,:] = GetGraphMetrics(G,labels)
        except:
            GM[i,j,:] = math.nan
GMD1 = np.zeros((len(CalcDatNAD1),5,5))
for i,C1 in enumerate(CalcDatNAD1):
    for j,C2 in enumerate(C1):
        Calc = C2[~np.isnan(C2.sum(axis=1))]
        Calc = Calc[Calc.sum(axis=1)>0]
        M = np.round(np.corrcoef(Calc),6)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'complete')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        GMD1[i,j,:] = GetGraphMetrics(G,labels)
GMD2 = np.zeros((len(CalcDatNAD2),5,5))
for i,C1 in enumerate(CalcDatNAD2):
    for j,C2 in enumerate(C1):
        Calc = C2[~np.isnan(C2.sum(axis=1))]
        Calc = Calc[Calc.sum(axis=1)>0]
        M = np.corrcoef(Calc)
        np.fill_diagonal(M,0)
        M[abs(M)<0.2] = 0
        G = nx.from_numpy_array(M)
        hierarchy = linkage(pdist(M), 'ward')
        labels = fcluster(hierarchy,4, criterion='maxclust')
        GMD2[i,j,:] = GetGraphMetrics(G,labels)

In [ ]:
fig,axs = plt.subplots(1,4,figsize=(28,4))
for j,i in enumerate([0,1,3,4]):
    ax = axs[j]
    ax.errorbar([1,2,3,4],np.nanmean(GM[labelsMPC==2,:4,i],axis=0),stats.sem(GM[labelsMPC==1,:4,i],axis=0,nan_policy='omit'),
                 lw=3,label='R+',c=ColorCode['D1Nac'])
    ax.errorbar([1,2,3,4],np.nanmean(GM[labelsMPC==1,:4,i],axis=0),stats.sem(GM[labelsMPC==2,:4,i],axis=0,nan_policy='omit'),
             lw=3,label='R-',c=ColorCode['D1Nac']+0.3)
    ax.set_ylabel(FigLabs[i],fontsize=32)
    ax.set_xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
    pval_old = stats.ttest_ind(GM[labelsMPC==1,:-1,i],GM[labelsMPC==2,:-1,i],nan_policy='omit').pvalue
    try:
        x = np.copy(pval_old)
        reject_bh, pval, _, _ = multipletests(pval_old[pval_old<0.05], alpha=0.05, method='fdr_bh')
        x[pval_old<0.05] = pval
        pval = np.copy(x)
    except:
        pval = np.copy(pval_old)
    hval = np.nanmean(GM[labelsMPC==1,:-1,i],axis=0)+stats.sem(GM[labelsMPC==1,:-1,i],axis=0,nan_policy='omit')+np.nanmean(GM[:,:-1,i],axis=0)*0.1

    m1 = (pval<0.05)*(pval>0.01)
    ax.plot(np.array([1,2,3,4])[m1],hval[m1],marker=(6, 2,0 ),ms=10,c='k',ls='')
    m2 = pval<0.01
    ax.plot(np.array([1,2,3,4])[m2]-0.02,hval[m2],marker=(6, 2,0 ),ms=10,c='k',ls='')
    ax.plot(np.array([1,2,3,4])[m2]+0.02,hval[m2],marker=(6, 2,0 ),ms=10,c='k',ls='')
    m3 = pval<0.0001
    ax.plot(np.array([1,2,3,4])[m3]-0.06,hval[m3],marker=(6, 2,0 ),ms=10,c='k',ls='')
    ax.plot(np.array([1,2,3,4])[m3]+0.06,hval[m3],marker=(6, 2,0 ),ms=10,c='k',ls='')
    print(pval)
    ax.legend(fontsize=24,frameon=False,loc=1,ncols=2,bbox_to_anchor=(0.9,1.2))
    ax.spines["right"].set_visible(False)
    ax.spines["top"].set_visible(False) 

In [ ]:
fig,axs = plt.subplots(1,4,figsize=(28,4))
for j,i in enumerate([0,1,3,4]):
    ax = axs[j]
    A= GMD1[labelsNAD1==2,:4,i]
    B = GMD1[labelsNAD1==1,:4,i]
    ax.errorbar([1,2,3,4],np.nanmean(A,axis=0),stats.sem(A,axis=0,nan_policy='omit'),
                 lw=3,label='R+',c=ColorCode['D2Nac'])
    ax.errorbar([1,2,3,4],np.nanmean(B,axis=0),stats.sem(B,axis=0,nan_policy='omit'),
             lw=3,label='R-',c=np.clip(ColorCode['D2Nac']+0.2,0,1))
    ax.set_ylabel(FigLabs[i],fontsize=32)
    ax.set_xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
    pval = stats.ttest_ind(A,B,nan_policy='omit').pvalue
    hval = np.nanmean(GM[labelsMPC==1,:-1,i],axis=0)+stats.sem(GM[labelsMPC==1,:-1,i],axis=0,nan_policy='omit')+np.nanmean(GM[:,:-1,i],axis=0)*0.1
    ax.plot(np.array([1,2,3,4])[pval<0.05],hval[pval<0.05],marker=(6, 2,0 ),ms=10,c='k',ls='')
    ax.legend(fontsize=24,frameon=False,loc=1,ncols=2,bbox_to_anchor=(0.9,1.2))
    ax = plt.gca()
    ax.spines["right"].set_visible(False)

In [ ]:
fig,axs = plt.subplots(1,4,figsize=(28,4))
for j,i in enumerate([0,1,3,4]):
    ax = axs[j]
    A= GMD2[labelsNAD2==2,:4,i]
    B = GMD2[labelsNAD2==1,:4,i]
    ax.errorbar([1,2,3,4],np.nanmean(A,axis=0),stats.sem(A,axis=0,nan_policy='omit'),
                 lw=3,label='R+',c=ColorCode['mPFC'])
    ax.errorbar([1,2,3,4],np.nanmean(B,axis=0),stats.sem(B,axis=0,nan_policy='omit'),
             lw=3,label='R-',c=np.clip(ColorCode['mPFC']+0.2,0,1))
    ax.set_ylabel(FigLabs[i],fontsize=32)
    ax.set_xticks([1,2,3,4],['CFC','CMR1','CMR2','CMR3'])
    pval = stats.ttest_ind(A,B,nan_policy='omit').pvalue
    hval = np.nanmean(GM[labelsMPC==1,:-1,i],axis=0)+stats.sem(GM[labelsMPC==1,:-1,i],axis=0,nan_policy='omit')+np.nanmean(GM[:,:-1,i],axis=0)*0.1
    ax.plot(np.array([1,2,3,4])[pval<0.05],hval[pval<0.05],marker=(6, 2,0 ),ms=10,c='k',ls='')
    ax.legend(fontsize=24,frameon=False,loc=1,ncols=2,bbox_to_anchor=(0.9,1.2))
    ax = plt.gca()
    ax.spines["right"].set_visible(False)